# <center>CNN 2D for image classification<center>

Firstly:<br>

<ol>
  <li>Go to Edit
  <li>Notebook settings
  <li>On hardware accelerator, set GPU
  <li>Save
</ol>

In this way we are able to use the free GPU available on Google Colab to train our model

## Load drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

import sys
sys.path.append('/content/drive/MyDrive/IV')

## Load libraries

In [2]:
# Install missing packages

# Libraries
import os
import glob
import cv2
import random
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from utils import CustomDataset, compute_metrics, plot_weights, visTensor

# Import the model
from CNN_128x128 import CNN_128x128

# Style for chart
sns.set_style('darkgrid')
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=13)
plt.rc('ytick', labelsize=13)
plt.rc('legend', fontsize=13)
plt.rc('font', size=13)

### Load data

labels:
* dogs = 0
* flowers = 1

In [3]:
# Define train and test labels
train_labels = np.zeros(2400)
train_labels[1200:2400] = 1
test_labels = np.zeros(800)
test_labels[400:800] = 1
train_labels = train_labels.astype('uint8')
test_labels = test_labels.astype('uint8')

In [13]:
# Load train set
train_data = [cv2.imread(file) for file in glob.glob('./Data/train/dog/*.jpg')]
train_data.extend(cv2.imread(file) for file in glob.glob('./Data/train/flower/*.jpg'))

# Load test set
test_data = [cv2.imread(file) for file in glob.glob('./Data/test/dog/*.jpg')]
test_data.extend(cv2.imread(file) for file in glob.glob('./Data/test/flower/*.jpg'))

In [14]:
# Random shuffle train and test set
train_list = list(zip(train_data,train_labels))
test_list = list(zip(test_data,test_labels))

random.shuffle(train_list)
random.shuffle(test_list)

train_data, train_labels = zip(*train_list)
test_data, test_labels = zip(*test_list)

### Create datasets for train and test & Define useful variables for deep learning model

In [15]:
# Set device where to run the model. GPU if available, otherwise cpu (very slow with deep learning models)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device: ',device)

# Create Dataloader with batch size = 64
train_dataset = CustomDataset(train_data,train_labels)    # we use a custom dataset defined in utils.py file
test_dataset = CustomDataset(test_data,test_labels)       # we use a custom dataset defined in utils.py file

batch_size = 64

trainset = DataLoader(train_dataset,batch_size=batch_size,drop_last=True)    # construct the trainset with subjects divided in mini-batch
testset = DataLoader(test_dataset,batch_size=batch_size,drop_last=True)      # construct the testset with subjects divided in mini-batch

Device:  cpu


In [16]:
# Define useful variables
models_trained_path = '/content/drive/MyDrive/IV/models_trained/'
if not os.path.exists(models_trained_path):                 # create a directory where to save the best model
    os.makedirs(models_trained_path)

best_acc = 0.0
num_epochs = 40                                   # number of epochs
lr = 0.01                                         # learning rate
n_classes = len(np.unique(train_labels))                # number of classes in the dataset
lab_classes = ['Dog','Flower']

# Variables to store the resuts
losses = []
acc_train = []
pred_label_train = torch.empty((0)).to(device)    # .to(device) to move the data/model on GPU or CPU (default)
true_label_train = torch.empty((0)).to(device)

# Model
model = CNN_128x128(input_channel=3,num_classes=n_classes).to(device)

# Optimizer
optim = torch.optim.SGD(model.parameters(),lr = lr, momentum=0.5)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

### Train the 1D CNN to classify ECG data

In [ ]:
for epoch in range(num_epochs):
    # Train step
    model.train()                                                   # tells to the model you are in training mode (batchnorm and dropout layers work)
    for data_tr in trainset:
        optim.zero_grad()
        X_tr,y_tr = data_tr                                         # unlist the data from the train set
        X_tr = X_tr.view(batch_size,3,128,128).float().to(device)     # change the size for the input data - convert to float type
        y_tr = y_tr.to(device)
        output = model(X_tr)                                        # run the model
        loss = criterion(output,y_tr)                               # compute loss
        _,pred = output.max(1)                                      # get the index == class of the output along the rows (each sample)
        pred_label_train = torch.cat((pred_label_train,pred),dim=0)
        true_label_train = torch.cat((true_label_train,y_tr),dim=0)
        loss.backward()                                             # compute backpropagation
        optim.step()                                                # parameter update

    losses.append(loss.cpu().detach().numpy())
    acc_t = accuracy_score(true_label_train.cpu(),pred_label_train.cpu())
    acc_train.append(acc_t)
    print("  epoch : {}/{}, loss = {:.4f} - acc = {:.4f}".format(epoch + 1, num_epochs, loss, acc_t))
    if acc_t > best_acc:                                                            # save the best model (the highest accuracy in validation)
        torch.save(model.state_dict(),models_trained_path+'CNN_128x128_best_model_trained.pt')
        best_acc = acc_t

    # Reinitialize the variables to compute accuracy
    pred_label_train = torch.empty((0)).to(device)
    true_label_train = torch.empty((0)).to(device)


  epoch : 1/40, loss = 0.5702 - acc = 0.5853
  epoch : 2/40, loss = 0.5304 - acc = 0.6719
  epoch : 3/40, loss = 0.5239 - acc = 0.6981
  epoch : 4/40, loss = 0.4931 - acc = 0.7116
  epoch : 5/40, loss = 0.4823 - acc = 0.7259
  epoch : 6/40, loss = 0.4930 - acc = 0.7323
  epoch : 7/40, loss = 0.4918 - acc = 0.7492
  epoch : 8/40, loss = 0.4727 - acc = 0.7475
  epoch : 9/40, loss = 0.4887 - acc = 0.7399
  epoch : 10/40, loss = 0.4980 - acc = 0.7513
  epoch : 11/40, loss = 0.4846 - acc = 0.7627
  epoch : 12/40, loss = 0.5099 - acc = 0.7690
  epoch : 13/40, loss = 0.5375 - acc = 0.7546
  epoch : 14/40, loss = 0.5030 - acc = 0.7614
  epoch : 15/40, loss = 0.6094 - acc = 0.7753
  epoch : 16/40, loss = 0.4833 - acc = 0.7728
  epoch : 17/40, loss = 0.4835 - acc = 0.7808
  epoch : 18/40, loss = 0.4836 - acc = 0.7893
  epoch : 19/40, loss = 0.5702 - acc = 0.7876
  epoch : 20/40, loss = 0.4679 - acc = 0.8015
  epoch : 21/40, loss = 0.4738 - acc = 0.7973
  epoch : 22/40, loss = 0.5693 - acc = 0.79

In [ ]:
# Plot the results
plt.figure(figsize=(8,5))
plt.plot(list(range(num_epochs)), losses)
plt.title("Learning curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,5))
plt.plot(list(range(num_epochs)), acc_train)
plt.title("Accuracy curve")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.show()

### Test the trained model

In [ ]:
model_test = CNN_128x128(input_channel=3,num_classes=n_classes).to(device)                # Initialize a new model
model_test.load_state_dict(torch.load(models_trained_path+'CNN_128x128_best_model_trained.pt'))   # Load the model

pred_label_test = torch.empty((0,n_classes)).to(device)
true_label_test = torch.empty((0)).to(device)

with torch.no_grad():
  for data in testset:
    X_te, y_te = data
    X_te = X_te.view(batch_size,3,128,128).float().to(device)
    y_te = y_te.to(device)
    output_test = model_test(X_te)
    pred_label_test = torch.cat((pred_label_test,output_test),dim=0)
    true_label_test = torch.cat((true_label_test,y_te),dim=0)

compute_metrics(y_true=true_label_test,y_pred=pred_label_test,lab_classes=lab_classes)    # function to compute the metrics (accuracy and confusion matrix)


### Visualize the kernels

In [ ]:
# Long to compute for layer 3 and 4
plot_weights(model_test.conv1, single_channel = False, collated = True)
# plot_weights(model_test.conv2, single_channel = True, collated = False)
# plot_weights(model_test.conv3, single_channel = True, collated = False)
# plot_weights(model_test.conv4, single_channel = True, collated = False)


In [ ]:
# Get the first kernel from the model
kernels_1 = model_test.conv1.weight.data.cpu().clone()
visTensor(kernels_1, ch=1, allkernels=False)
plt.axis('off')
plt.title('kernels from convolutional layer: 1')
plt.ioff()
plt.show()

# Get the second kernel from the model
kernels_2 = model_test.conv2.weight.data.cpu().clone()
visTensor(kernels_2, ch=0, allkernels=False)
plt.axis('off')
plt.title('kernels from convolutional layer: 2')
plt.ioff()
plt.show()

# Get the second kernel from the model
kernels_3 = model_test.conv3.weight.data.cpu().clone()
visTensor(kernels_3, ch=0, allkernels=False)
plt.axis('off')
plt.title('kernels from convolutional layer: 3')
plt.ioff()
plt.show()

# Get the second kernel from the model
kernels_4 = model_test.conv4.weight.data.cpu().clone()
visTensor(kernels_4, ch=0, allkernels=False)
plt.axis('off')
plt.title('kernels from convolutional layer: 4')
plt.ioff()
plt.show()



### Visualize features map

In [ ]:
conv_weights =[]                            # save the weights of convolutional layers
conv_layers = []                            # save the convolutional layers
model_children = list(model.children())     # get all the model children as list
# append all the convolutional layers and their respective wights to the list
for i in range(len(model_children)):
    if type(model_children[i]) == torch.nn.Conv2d:
        conv_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])

print(conv_layers)

In [ ]:
image = X_te[2,:,:,:]
original_image = image


In [ ]:
# process the images through all the convolutional layers 
outputs = []
names = []
for layer in conv_layers[0:]:   # run over the convolutional layers
    image = layer(image)        # process the image
    outputs.append(image)       # save the output of the layer 
    names.append(str(layer))    # save the name of the layer
print(len(outputs))

# print feature_maps
for feature_map in outputs:
    print(feature_map.shape)

In [ ]:
# Convert from 3D to 2D summing the element for each channel
processed = []
for feature_map in outputs:
    feature_map = feature_map.squeeze(0)
    gray_scale = torch.sum(feature_map,0)
    gray_scale = gray_scale / feature_map.shape[0]
    processed.append(gray_scale.data.cpu().numpy())
for fm in processed:
    print(fm.shape)

In [ ]:
fig = plt.figure(figsize=(30, 50))
for i in range(len(processed)):
    a = fig.add_subplot(5, 4, i+1)
    imgplot = plt.imshow(processed[i],cmap='viridis')
    a.axis("off")
    a.set_title(names[i].split('(')[0], fontsize=30)

plt.show()

In [ ]:
plt.figure()
plt.imshow(original_image.reshape(128,128,3).cpu().detach().numpy().astype('uint8'))
plt.axis('off')
plt.show()